League of Legends is a popular Multiplayer Online Battle Arena (MOBA) that is played by more than 100 Million people around the globe. Data is publicly available and can be accessed. The problem is that the data structure is very incovenient (a lot of dictionaries) and a lot of data wrangling is necessary until actual, useful questions can be answered. This project gets data from the Riot Games API.

In [1]:
import pandas as pd


Teams try to find talented players. We want to get stats for the best players (Rank: Challenger) of the Korean server. Teams' scouts might be interested in metrics.


Reading in data from Oracle's Elixir, downloaded on 1-13-2021.

In [2]:
sp_2021_reg, sp_2021_pl, su_2021_reg, su_2021_pl =  pd.read_csv("LCK CL 2021 Spring - Player Stats - OraclesElixir.csv"), pd.read_csv("LCK CL 2021 Spring Playoffs - Player Stats - OraclesElixir.csv"), pd.read_csv("LCK CL 2021 Summer - Player Stats - OraclesElixir.csv"), pd.read_csv("LCK CL 2021 Summer Playoffs - Player Stats - OraclesElixir.csv")

all_splits = [sp_2021_reg, sp_2021_pl, su_2021_reg, su_2021_pl]

all_splits_df = pd.concat(all_splits)

Inspecting one players as an example.

In [3]:
all_splits_df.query('Player == "Berserker"').columns

Index(['Player', 'Team', 'Pos', 'GP', 'W%', 'CTR%', 'K', 'D', 'A', 'KDA', 'KP',
       'KS%', 'DTH%', 'FB%', 'GD10', 'XPD10', 'CSD10', 'CSPM', 'CS%P15', 'DPM',
       'DMG%', 'D%P15', 'EGPM', 'GOLD%', 'STL', 'WPM', 'WCPM'],
      dtype='object')

We want to change some column names to avoid confusion.

In [4]:
all_splits_df = all_splits_df.rename(columns={"K": "total_kills",
"Pos": "position",
 "D": "total_deaths",
  "A": "total_assists",
  "GP": "games_played",
   "W%": "winrate%",
    "CTR%": "pick_after_opponent%",
     "KP":"kill_assist_participation%",
      "KS%":"kill_participation%",
      "DTH%":"average_share_teams_deaths%",
      "FB%": "first_blood_participation_rate%",
      "GD10": "average_gold_diff_10_min",
      "XPD10": "average_xp_diff_10_min",
      "CSD10": "average_cs_diff_10_min",
      "CSPM": "average_CSPM",
      "CS%P15": "average_share_team_cs_post_15_min%",
      "DPM": "average_DPM_to_champs",
      "DMG%": "average_share_team_DMG_to_champs%",
      "D%P15": "average_share_team_DMG_to_champs_post_15_min%",
      "EGPM": "average_gold_per_min",
      "GOLD%": "average_share_team_gold%",
      "STL": "num_neutral_obj_stolen",
      "WPM": "average_wards_per_min",
      "WCPM": "average_wards_cleared_per_min"})

Inspecting the resulting column names and their datatypes. 

In [5]:
all_splits_df.dtypes

Player                                            object
Team                                              object
position                                          object
games_played                                       int64
winrate%                                          object
pick_after_opponent%                              object
total_kills                                        int64
total_deaths                                       int64
total_assists                                      int64
KDA                                              float64
kill_assist_participation%                        object
kill_participation%                               object
average_share_teams_deaths%                       object
first_blood_participation_rate%                   object
average_gold_diff_10_min                           int64
average_xp_diff_10_min                             int64
average_cs_diff_10_min                           float64
average_CSPM                   

All columns that are measured in percent are not recognized as numeric columns, but as object columns. In order to use mathematical operations with these columns, they have to be transformed to numeric columns.

In [6]:
all_splits_df['winrate%'].str.rstrip('%')

#all_splits_df.astype({'winrate%' : 'float64'})

0     55
1     83
2     63
3     83
4     42
      ..
28    40
29    40
30    44
31    50
32    50
Name: winrate%, Length: 195, dtype: object

As we have seen above, some players participated in more than one split, we have to aggregate their data, such that we end up with one row for every player. We want to keep non-performance related features, as they are in the raw data: this includes the player name, his team, his position and the number of games played. Some metrics allow for just averaging over them (e.g. all averaged metrics). This is possible, as these metrics refer to an average game of a player in one split, i.e. playoff. Thus, taking their mean just means that the value now represents the average game of all tournaments, a player took part in. However, this can not be done for absolute measures who refer to a whole tournaments, such as kills. This is due to the fact that certain players have participated in different tournaments that had a different amount of games. This questions the general use of absolute measures.

We are gonna create lists that specify columns that will be aggregated in the same way: one list for the columns that will be summed, one list for the columns that will be averaged, and finally, one list for the remaining, non-numerical, columns.

In [7]:
cols_to_sum = ['games_played',
 'total_kills', 'total_deaths',
  'total_assists', 'num_neutral_obj_stolen']

cols_to_keep = ['Player','Team', 'position']

cols_to_avg = [col for col in all_splits_df.columns if col not in (cols_to_sum + cols_to_keep)]

Transforming string columns that are stored in % to numerical columns, before grouping and performing the aggregation.

In [8]:
percentage_cols = [col for col in cols_to_avg if col.endswith("%")]

all_splits_df_percentage_cols = all_splits_df[percentage_cols].astype(str).applymap(lambda x: x.strip("%"))



all_splits_df = pd.concat([all_splits_df.drop(percentage_cols, axis = 1),
 all_splits_df_percentage_cols.astype('float16')], axis = 1)

In [9]:
all_splits_df.dtypes

Player                                            object
Team                                              object
position                                          object
games_played                                       int64
total_kills                                        int64
total_deaths                                       int64
total_assists                                      int64
KDA                                              float64
average_gold_diff_10_min                           int64
average_xp_diff_10_min                             int64
average_cs_diff_10_min                           float64
average_CSPM                                     float64
average_DPM_to_champs                              int64
average_gold_per_min                               int64
num_neutral_obj_stolen                           float64
average_wards_per_min                            float64
average_wards_cleared_per_min                    float64
winrate%                       

Grouping and aggregating.

In [10]:
all_splits_by_player = all_splits_df.groupby(['Player'])

orig_cols = all_splits_df[cols_to_keep].drop_duplicates('Player')
summed_cols = all_splits_by_player[cols_to_sum].sum()


avg_cols = all_splits_by_player[cols_to_avg].mean()



 Concatenating the dfs. 

In [11]:
orig_summed_cols = pd.concat([orig_cols.set_index('Player'), summed_cols], axis = 1)
agg_all = pd.concat([orig_summed_cols, avg_cols], axis = 1)